# Η θεωρία: χαρακτήρες και κωδικοποιήσεις

* Η πληροφορία σε μορφή **απλού κειμένου** (plain text) αποτελείται από **χαρακτήρες** (characters).
* Οι χαρακτήρες αναπαρίστανται σε κάποιο πρότυπο (όπως το Unicode) μέσω **αριθμών** (code points). Οι αριθμοί αυτοί στο Unicode είναι **μεγαλύτεροι από 1 byte**.
* Όταν έρθει η ώρα αποθήκευσης σε αρχείο, οι αριθμοί πρέπει να γίνουν **σειρές από bytes**: η διαδικασία ονομάζεται **κωδικοποίηση**.

## Πώς χειρίζονται τους χαρακτήρες οι εφαρμογές
1. Μια μοντέρνα εφαρμογή διαβάζει το κείμενο ως bytes από αρχείο
2. Το αναπαριστά στη μνήμη ανάλογα με τη γλώσσα προγραμματισμού (**αποκωδικοποίηση**)
3. Το επεξεργάζεται 
4. Στο τέλος το αποθηκεύει ως bytes σε αρχείο (**κωδικοποίηση**)

# Κωδικοποίηση UTF-8

Η κωδικοποίηση χαρακτήρων Unicode συνήθως γίνεται σήμερα στη μορφή **UTF-8**. Πρόκειται για μορφή με **μεταβλητό αριθμό bytes ανά χαρακτήρα**: η ιδέα είναι να έχετε λίγα bytes για τους περισσότερο χρησιμοποιούμενους χαρακτήρες (δηλ. ascii) και περισσότερα bytes για λιγότερο χρησιμοποιούμενους χαρακτήρες.

Η κωδικοποίηση ορίζεται από τον εξής πίνακα:

```
00...7F 
0xxxxxxx 

80...7FF 
110xxxxx  10xxxxxx 

800...FFFF 
1110xxxx  10xxxxxx  10xxxxxx 

10000...10FFFF 
11110xxx  10xxxxxx  10xxxxxx  10xxxxxx 
```
*(διαβάστε τον πίνακα ως εξής: "αν ο αριθμός code point του χαρακτήρα είναι μεταξύ 00 και 7F, η κωδικοποίηση γίνετα με ένα byte στη μορφή 0xxxxxxx" κλπ. Κάθε x κρατάει ένα bit του code point)*

Καθώς το UTF-8 ορίζει ρητά τη σειρά αποθήκευσης, δεν υπάρχουν εδώ προβλήματα **endian-ness**.

# Δοκιμές στην Python3
Οι μοντέρνες γλώσσες προγραμματισμού χειρίζονται την κωδικοποίηση και την αποκωδικοποίηση UTF-8 (κι όχι μόνο αυτήν) για εμάς. Ας δούμε κάποια παραδείγματα στην Python3.

Στην Python3 ο κύριος τύπος κειμενικών δεδομένων είναι το **string**: αυτό μπορείτε να το δείτε ως μία ακολουθία χαρακτήρων Unicode. Ας τυπώσουμε το code point κάθε χαρακτήρα στο string `'Άρτα 2'`

In [11]:
s = 'Άρτα 2'

for c in s:
    print(hex(ord(c)))

0x386
0x3c1
0x3c4
0x3b1
0x20
0x32


*Σημ: Η συνάρτηση `ord()` επιστρέφει τον αριθμό code point ενός χαρακτήρα ενώ η `hex()` μετατρέπει τον αριθμό σε δεκαεξαδική μορφή*

Ας **κωδικοποιήσουμε** το string `s` κατά UTF-8. Η μεταβλητή `b` που προκύπτει είναι τύπου **bytes**: δείτε τον τυπο αυτό ως ακολουθία bytes (0..255).

In [12]:
b = s.encode('utf-8')
print('Encoding has {} bytes'.format(len(b)))
for value in b:
    print(hex(value))

Encoding has 10 bytes
0xce
0x86
0xcf
0x81
0xcf
0x84
0xce
0xb1
0x20
0x32


Στο προηγούμενο παράδειγμα, η κωδικοποίηση UTF-8 που προκύπτει αποτελείται από 10 bytes: 4x2 για τους τέσσερις ελληνικούς χαρακτήρες και 2 ακόμα για τους ascii χαρακτήρες κενό (space) και  '2'. 

Μπορούμε να ζητήσουμε και άλλες κωδικοποιήσεις: ας χρησιμοποιήσουμε το παλιό `iso-8859-7` (ελληνικά, 1 byte ανά χαρακτήρα):

In [13]:
b = s.encode('iso-8859-7')
print('Encoding has {} bytes'.format(len(b)))
for value in b:
    print(hex(value))

Encoding has 6 bytes
0xb6
0xf1
0xf4
0xe1
0x20
0x32


Το πρόβλημα με τις κωδικοποιήσεις του ενός byte ανά χαρακτήρα είναι ότι δεν μπορούμε να έχομε πολυγλωσσικά κείμενα.

Αν **αποκωδικοποιήσουμε** με λάθος πρότυπο, το αποτέλεσμα θα είναι ευτράπελο: 

In [14]:
s2 = b.decode('iso-8859-1')
print(s2)

¶ñôá 2


*Σίγουρα θα έχετε δει κάποια φορά κάτι σαν το προηγούμενο  ;-)*

# Παράδειγμα αποκωδικοποίησης
Θα θυμάστε την άσκηση που κάναμε στο μάθημα "στο χέρι":

Έχοντας την κωδικοποιημένη κατά UTF-8 σειρά bytes (σε δεκαεξαδικό): 

```66 cf 85 72 e5 ad 9a ce ba``` 

i) Πόσους unicode χαρακτήρες περιέχει η πιο πάνω σειρά bytes;

ii) Ποιοι οι unicode χαρακτήρες αυτοί; (σε δεκαεξαδική μορφή)

Ας ζητήσουμε από την Python να το κάνει για εμάς!

In [15]:
b = b'\x66\xcf\x85\x72\xe5\xad\x9a\xce\xba'

s = b.decode('utf-8')
for c in s:
    print('{}\t{}'.format(c,hex(ord(c))))


f	0x66
υ	0x3c5
r	0x72
孚	0x5b5a
κ	0x3ba


*Σημ: το* 
```python
b'\x66\xcf\x85\x72\xe5\xad\x9a\xce\xba'
```
*είναι η σύνταξη της Python3 για μια ακολουθία bytes που περιέχει τα 66, cf, 85, 72, e5, ad, 9a, ce, ba*

# Γιατί δεν φαίνονται;
Σε ένα κείμενο περιέχονται τα μαθηματικά καλλιγραφικά 𝒜ℒ𝒞. Σε κάποιες συσκευές ο πρώτος και ο τελευταίος χαρακτήρας δεν φαίνονται (ίσως δεν τους βλέπετε κι εσείς, ανάλογα με τη συσκευή που χρησιμοποιείτε τώρα!). Γιατί άραγε;

Ας δούμε τα code points:

In [16]:
s ='𝒜ℒ𝒞'
for c in s:
    print(hex(ord(c)))

0x1d49c
0x2112
0x1d49e


Αχά! Ο πρώτος και ο τελευταίος χαρακτήρας έχουν code points > 0xFFFF!

Η περιοχή χαρακτήρων με την μεγαλύτερη υποστήριξη ανήκει στην περιοχή 0...0xFFFF, η οποία ονομάζεται και **BMP (Basic Multilingual Plane)** του Unicode. Αν οι χαρακτήρες είναι έξω από την περιοχή αυτή (όπως ο 0x1d49c και ο 0x1d49e), τότε ίσως εμφανιστούν προβλήματα απεικόνισης..

*Σημ: το προηγούμενο πρόβλημα δεν οφείλεται στο code point. Οι σύγχρονες συσκευές και εφαρμογές μπορούν να χειριστούν άνετα οποιονδήποτε unicode χαρακτήρα. Δεν ισχύει όμως το ίδιο με τις διαθέσιμες γραμματοσειρές (fonts)!*